# Calculating the Dates for Milestones  from the Google Sheet Used to Populate Them

- 09/16/21
- James M. Irving

# Goal

- To convert the curriculum template of base milestones into dataframes that are dynamically calculated based on launch data.

- To jump to the section using the completed functions, click [here](#workflow)

In [1]:
raise Exception('You may want to jump to the Full Workflow header linked above instead of running all.')

Exception: ignored

## Load in Data & Explore 

In [32]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
# gsheet_name = 'JMI Copy of 2021-07-15 Flex Milestones Template'
gsheet_name = "2021-07-15 Flex Milestones Template"
file = gc.open(gsheet_name)
file

<Spreadsheet '2021-07-15 Flex Milestones Template' id:1UIbNBMOCcsFKI4LSUNAxwdlVqaCz3WtUALDjeujxYwI>

In [33]:
if len(file.worksheets())==1:
    worksheet = file.get_worksheet(0)
else:
    print(file.worksheets())
    val = input("select which sheet number to use")
    worksheet = file.get_worksheet(int(val))
worksheet

<Worksheet 'v2.2' id:576321280>

In [34]:

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[2:],columns=rows[1])


display(df.info(),df.head())
print(df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Week                     145 non-null    object
 1   Weekday                  145 non-null    object
 2   Day # (20wk)             145 non-null    object
 3   Canvas Topics            145 non-null    object
 4   Milestone                145 non-null    object
 5   Duration in Base (20wk)  145 non-null    object
 6   Duration in Base (40wk)  145 non-null    object
 7   Duration in Base (60wk)  145 non-null    object
 8   Checkpoints (Quizzes)    145 non-null    object
 9   Blog Posts               145 non-null    object
 10  Code Challenges          145 non-null    object
 11  Projects                 145 non-null    object
dtypes: object(12)
memory usage: 13.7+ KB


None

,Week,Weekday,Day # (20wk),Canvas Topics,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Checkpoints (Quizzes),Blog Posts,Code Challenges,Projects
0,1,Monday,0,Topic 0,,0,0,0,,,,
1,1,Tuesday,1,Topic 1,,0,0,0,,,,
2,1,Wednesday,2,Topic 2,,0,0,0,,,,
3,1,Thursday,3,Topic 3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,,,,
4,1,Friday,4,,,0,0,0,Data Serialization Formats Checkpoint,,,


Index(['Week', 'Weekday', 'Day # (20wk)', 'Canvas Topics', 'Milestone',
       'Duration in Base (20wk)', 'Duration in Base (40wk)',
       'Duration in Base (60wk)', 'Checkpoints (Quizzes)', 'Blog Posts',
       'Code Challenges', 'Projects'],
      dtype='object')


### Cleaning data

In [35]:
## rename day # and remove total rows
df = df.rename({'Day # (20wk)':'Day #'},axis=1)
df = df.loc[ ~df['Day #'].str.contains('total',case=False)]
df

,Week,Weekday,Day #,Canvas Topics,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Checkpoints (Quizzes),Blog Posts,Code Challenges,Projects
0,1,Monday,0,Topic 0,,0,0,0,,,,
1,1,Tuesday,1,Topic 1,,0,0,0,,,,
2,1,Wednesday,2,Topic 2,,0,0,0,,,,
3,1,Thursday,3,Topic 3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,,,,
4,1,Friday,4,,,0,0,0,Data Serialization Formats Checkpoint,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
139,20,Wednesday,135,,,0,0,0,,,,
140,20,Thursday,136,,,0,0,0,,,,
141,20,Friday,137,,Capstone Project Review,11,22,33,,,,Capstone Project Review
142,20,Saturday,138,,,0,0,0,,,,


In [36]:
## saving lists of columns to keep (and to recast as numeric)
duration_cols =  [c for c in df.columns if 'duration' in c.lower()]
info_cols = ['Day #', 'Milestone']

cols_to_keep =info_cols +duration_cols

## saving needed cols
df = df[cols_to_keep]
df

,Day #,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk)
0,0,,0,0,0
1,1,,0,0,0
2,2,,0,0,0
3,3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12
4,4,,0,0,0
...,...,...,...,...,...
139,135,,0,0,0
140,136,,0,0,0
141,137,Capstone Project Review,11,22,33
142,138,,0,0,0


In [37]:
## convert to numeric 
for col in duration_cols:
    df[col] = pd.to_numeric(df[col])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 0 to 143
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Day #                    140 non-null    object
 1   Milestone                140 non-null    object
 2   Duration in Base (20wk)  140 non-null    int64 
 3   Duration in Base (40wk)  140 non-null    int64 
 4   Duration in Base (60wk)  140 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 6.6+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# df.insert(0,'Start Date',pd.to_datetime('08-09-2021'))
# df

In [39]:
df['Duration in Base (20wk)']

0       0
1       0
2       0
3       4
4       0
       ..
139     0
140     0
141    11
142     0
143     2
Name: Duration in Base (20wk), Length: 140, dtype: int64

### Approach
- I need to use the Duration Columns to be able to calcualte the date of each milestone. 

- Try:
    - Removing every row that isn't a milestone, 
    - Create cumulative sum columns 
    - Use start date + cumulative sums to calcualte milestone dates

In [40]:
df

,Day #,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk)
0,0,,0,0,0
1,1,,0,0,0
2,2,,0,0,0
3,3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12
4,4,,0,0,0
...,...,...,...,...,...
139,135,,0,0,0
140,136,,0,0,0
141,137,Capstone Project Review,11,22,33
142,138,,0,0,0


In [41]:
miles = df[ df['Duration in Base (20wk)']!=0].copy()
miles

,Day #,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk)
3,3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12
8,8,Topic 4: EDA with Pandas - Cumulative Lab,5,10,15
10,10,Topic 5: Pandas Data Cleaning - Cumulative Lab,2,4,6
14,14,Topic 7: SQL - Cumulative Lab,4,8,12
16,16,Topic 9: APIs - Cumulative Lab,2,4,6
25,25,Phase 1 Project,9,18,27
27,27,Phase 1 Blog Post,2,4,6
36,35,Topic 15: Hypothesis Testing - Cumulative Lab,8,16,24
38,37,Topic 16: Website A/B Testing - Cumulative Lab,2,4,6
45,44,Topic 19: Preprocessing with scikit-learn - Cu...,7,14,21


In [42]:
## Convert Duration Columns into New Cumulative Duration cols
cumul_cols = []
for col in duration_cols:
    new_col = f"Cumulative {col.replace('in Base ','')}"
    miles[new_col] = miles[col].cumsum()
    cumul_cols.append(new_col)

miles

,Day #,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Cumulative Duration (20wk),Cumulative Duration (40wk),Cumulative Duration (60wk)
3,3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,4,8,12
8,8,Topic 4: EDA with Pandas - Cumulative Lab,5,10,15,9,18,27
10,10,Topic 5: Pandas Data Cleaning - Cumulative Lab,2,4,6,11,22,33
14,14,Topic 7: SQL - Cumulative Lab,4,8,12,15,30,45
16,16,Topic 9: APIs - Cumulative Lab,2,4,6,17,34,51
25,25,Phase 1 Project,9,18,27,26,52,78
27,27,Phase 1 Blog Post,2,4,6,28,56,84
36,35,Topic 15: Hypothesis Testing - Cumulative Lab,8,16,24,36,72,108
38,37,Topic 16: Website A/B Testing - Cumulative Lab,2,4,6,38,76,114
45,44,Topic 19: Preprocessing with scikit-learn - Cu...,7,14,21,45,90,135


In [43]:
start_date = pd.to_datetime('08-09-2021')
print(start_date)

2021-08-09 00:00:00


In [44]:
col = cumul_cols[0]
col

'Cumulative Duration (20wk)'

In [45]:
pace = col.split('(')[-1].split(')')[0]
pace

'20wk'

In [46]:
## create milestone date cols
for col in cumul_cols:
    pace = col.split('(')[-1].split(')')[0] 
    miles[f"Milestone Date ({pace})"] = miles[col].map( lambda x: start_date + pd.Timedelta(f"{x} days"))

miles

,Day #,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Cumulative Duration (20wk),Cumulative Duration (40wk),Cumulative Duration (60wk),Milestone Date (20wk),Milestone Date (40wk),Milestone Date (60wk)
3,3,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,4,8,12,2021-08-13,2021-08-17,2021-08-21
8,8,Topic 4: EDA with Pandas - Cumulative Lab,5,10,15,9,18,27,2021-08-18,2021-08-27,2021-09-05
10,10,Topic 5: Pandas Data Cleaning - Cumulative Lab,2,4,6,11,22,33,2021-08-20,2021-08-31,2021-09-11
14,14,Topic 7: SQL - Cumulative Lab,4,8,12,15,30,45,2021-08-24,2021-09-08,2021-09-23
16,16,Topic 9: APIs - Cumulative Lab,2,4,6,17,34,51,2021-08-26,2021-09-12,2021-09-29
25,25,Phase 1 Project,9,18,27,26,52,78,2021-09-04,2021-09-30,2021-10-26
27,27,Phase 1 Blog Post,2,4,6,28,56,84,2021-09-06,2021-10-04,2021-11-01
36,35,Topic 15: Hypothesis Testing - Cumulative Lab,8,16,24,36,72,108,2021-09-14,2021-10-20,2021-11-25
38,37,Topic 16: Website A/B Testing - Cumulative Lab,2,4,6,38,76,114,2021-09-16,2021-10-24,2021-12-01
45,44,Topic 19: Preprocessing with scikit-learn - Cu...,7,14,21,45,90,135,2021-09-23,2021-11-07,2021-12-22


In [47]:
df_final = miles[[c for c in miles.columns if "milestone" in c.lower()]].copy()
df_final.reset_index(drop=True,inplace=True)
df_final

,Milestone,Milestone Date (20wk),Milestone Date (40wk),Milestone Date (60wk)
0,Topic 3: Data Serialization Formats - Cumulati...,2021-08-13,2021-08-17,2021-08-21
1,Topic 4: EDA with Pandas - Cumulative Lab,2021-08-18,2021-08-27,2021-09-05
2,Topic 5: Pandas Data Cleaning - Cumulative Lab,2021-08-20,2021-08-31,2021-09-11
3,Topic 7: SQL - Cumulative Lab,2021-08-24,2021-09-08,2021-09-23
4,Topic 9: APIs - Cumulative Lab,2021-08-26,2021-09-12,2021-09-29
5,Phase 1 Project,2021-09-04,2021-09-30,2021-10-26
6,Phase 1 Blog Post,2021-09-06,2021-10-04,2021-11-01
7,Topic 15: Hypothesis Testing - Cumulative Lab,2021-09-14,2021-10-20,2021-11-25
8,Topic 16: Website A/B Testing - Cumulative Lab,2021-09-16,2021-10-24,2021-12-01
9,Topic 19: Preprocessing with scikit-learn - Cu...,2021-09-23,2021-11-07,2021-12-22


### Functionizing Process

# Full Workflow <a name="workflow"></a>

In [48]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())



In [49]:
import pandas as pd

def open_sheet(gsheet_name = "2021-07-15 Flex Milestones Template"):

    file = gc.open(gsheet_name)

    ## load specific sheet 
    if len(file.worksheets())==1:
        worksheet = file.get_worksheet(0)
    else:
        print(file.worksheets())
        val = input("select which sheet number to use")
        worksheet = file.get_worksheet(int(val))

    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()

    # Convert to a DataFrame and render.
    import pandas as pd
    df = pd.DataFrame.from_records(rows[2:],columns=rows[1])

    return df


    
def get_miles_df(df,drop_cols=True):
    df = df.copy()
    ## rename day # and remove total rows
    df = df.rename({'Day # (20wk)':'Day #'},axis=1)
    df = df.loc[ ~df['Day #'].str.contains('total',case=False)]


    ## saving lists of columns to keep (and to recast as numeric)
    duration_cols =  [c for c in df.columns if 'duration' in c.lower()]
    info_cols = ['Milestone']

    if drop_cols==True:
        cols_to_keep =info_cols +duration_cols

        ## saving needed cols
        df = df[cols_to_keep]
    

        ## convert to numeric 
    for col in duration_cols:
        df[col] = pd.to_numeric(df[col])


    miles = df[ df['Duration in Base (20wk)']!=0].copy()
    ## Calculate cumulative cols
    cumul_cols = []
    for col in duration_cols:
        new_col = f"Cumulative {col.replace('in Base ','')}"
        miles[new_col] = miles[col].cumsum()
        cumul_cols.append(new_col)


    miles = miles.reset_index(drop=True)
    miles.index = miles.index+1

    return miles




def calc_milestone_dates(miles, start='08-09-2021'):
    miles = miles.copy()
    duration_cols =  [c for c in miles.columns if 'duration' in c.lower()]    
    start_date = pd.to_datetime(start)
    cumul_cols = [c for c in miles.columns if 'cumulative' in c.lower()]

        
    ## create milestone date cols
    for col in cumul_cols:
        pace = col.split('(')[-1].split(')')[0] 
        miles[f"Milestone Date ({pace})"] = miles[col].map( lambda x: start_date + pd.Timedelta(f"{x} days"))


    df_final = miles[[c for c in miles.columns if "milestone" in c.lower()]].copy()

    ## Setting the index as the milestone #
    df_final = df_final.reset_index(drop=True)
    df_final.index = df_final.index+1

    ## insert start date

    start_row = pd.Series({'Milestone':	'Start Date' ,
                       "Milestone Date (20wk)":start_date,
           "Milestone Date (40wk)":start_date,
           "Milestone Date (60wk)":start_date})

    df_final.loc[0] = start_row
    # df_final
    return df_final.sort_index()


In [50]:
## Get data 
df = open_sheet()
miles = get_miles_df(df,drop_cols=True)
miles.head(2)

,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Cumulative Duration (20wk),Cumulative Duration (40wk),Cumulative Duration (60wk)
1,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,4,8,12
2,Topic 4: EDA with Pandas - Cumulative Lab,5,10,15,9,18,27


In [51]:
## Calaculate Milestones
milestones = calc_milestone_dates(miles, start='08-30-21')
milestones

,Milestone,Milestone Date (20wk),Milestone Date (40wk),Milestone Date (60wk)
0,Start Date,2021-08-30,2021-08-30,2021-08-30
1,Topic 3: Data Serialization Formats - Cumulati...,2021-09-03,2021-09-07,2021-09-11
2,Topic 4: EDA with Pandas - Cumulative Lab,2021-09-08,2021-09-17,2021-09-26
3,Topic 5: Pandas Data Cleaning - Cumulative Lab,2021-09-10,2021-09-21,2021-10-02
4,Topic 7: SQL - Cumulative Lab,2021-09-14,2021-09-29,2021-10-14
5,Topic 9: APIs - Cumulative Lab,2021-09-16,2021-10-03,2021-10-20
6,Phase 1 Project,2021-09-25,2021-10-21,2021-11-16
7,Phase 1 Blog Post,2021-09-27,2021-10-25,2021-11-22
8,Topic 15: Hypothesis Testing - Cumulative Lab,2021-10-05,2021-11-10,2021-12-16
9,Topic 16: Website A/B Testing - Cumulative Lab,2021-10-07,2021-11-14,2021-12-22


### Looping through launch dates

In [52]:
## getting llaunch dates through 2022
launch_dates_range = pd.date_range(start='07-19-21',freq='3W-MON',periods=26)
launch_dates_range

DatetimeIndex(['2021-07-19', '2021-08-09', '2021-08-30', '2021-09-20',
               '2021-10-11', '2021-11-01', '2021-11-22', '2021-12-13',
               '2022-01-03', '2022-01-24', '2022-02-14', '2022-03-07',
               '2022-03-28', '2022-04-18', '2022-05-09', '2022-05-30',
               '2022-06-20', '2022-07-11', '2022-08-01', '2022-08-22',
               '2022-09-12', '2022-10-03', '2022-10-24', '2022-11-14',
               '2022-12-05', '2022-12-26'],
              dtype='datetime64[ns]', freq='3W-MON')

In [53]:
## Loop through launch dates to calculate milestones and save in dictionary
MILESTONES = {}
for launch in launch_dates_range:

    launch_str = launch.strftime('%m-%d-%y')
    # fname = save_fpath+f"flex-milestone-dates-{launch_str}-launch.csv"

    temp_df = calc_milestone_dates(miles, start=launch_str)
    MILESTONES[launch_str] = temp_df.copy()
    # temp_df.to_csv(fname,index=False)

## Saving Files

- First, just exporting a different csv for each launch date and using those in student meetings could be sufficient.

### Mount GDrive, Specify Folder

In [54]:
## Mount google drive to save excel
from google.colab import drive
import os



gdrive_path='gdrive/'
drive.mount(gdrive_path)


## Make folder for saving the csv documents
flex_folder = os.path.join(gdrive_path,"My Drive/",'Misc Documents/')
save_fpath = os.path.join(flex_folder,'Flex Milestone Dates - by Launch/')
os.makedirs(save_fpath,exist_ok=True)
print(save_fpath)

Drive already mounted at gdrive/; to attempt to forcibly remount, call drive.mount("gdrive/", force_remount=True).
gdrive/My Drive/Misc Documents/Flex Milestone Dates - by Launch/


In [55]:
## save files
def save_milestones(MILESTONES, save_fpath=None,save_as_excel = True,verbose=True):
    if save_fpath is None:
        save_fpath  = './'

    if save_as_excel == True:
        ## save to excel file as separate sheets
        excel_fname = save_fpath+'flex-milestones-by-launch-date.xlsx'
        # print(excel_fname)
        with pd.ExcelWriter(excel_fname,
                            datetime_format='mm-dd-yy',
                            date_format='mm-dd-yy') as writer:

            for launch_str, temp_df in MILESTONES.items():
                temp_df.to_excel(writer,sheet_name=launch_str,index=False)

        if verbose: print(f'\n[i] Milestones saved as "{excel_fname}".')


    else:

        if verbose: print(f'\n[i] Saving milestones as individualcsvs in {save_fpath}:')
        for launch_str, temp_df in MILESTONES.items():
                fname = save_fpath+f"flex-milestone-dates-{launch_str}-launch.csv"
                temp_df.to_csv(fname,index=False)
                if verbose: print(f"   - {fname.replace(save_fpath,'')}")

In [56]:
save_milestones(MILESTONES,save_fpath, save_as_excel=True)
save_milestones(MILESTONES,save_fpath, save_as_excel=False,verbose=False)


[i] Milestones saved as "gdrive/My Drive/Misc Documents/Flex Milestone Dates - by Launch/flex-milestones-by-launch-date.xlsx".


- While the function to calculate milestones is great and all, it doesn't fit into our one on one workflow. 
- I would like to export a sheet that is ready for a Google-Sheets calculator page. 


In [29]:
save_fpath

'gdrive/My Drive/Misc Documents/Flex Milestone Dates - by Launch/'

In [31]:
## Save a cumulative duration csv without dates
df = open_sheet()
miles = get_miles_df(df,drop_cols=True)

## Fix Index
miles.index.name = '#'
miles.reset_index(inplace=True,drop=False)
miles.to_csv(save_fpath+'Flex Milestones Durations - Cumulative.csv',index=False)
miles

,#,Milestone,Duration in Base (20wk),Duration in Base (40wk),Duration in Base (60wk),Cumulative Duration (20wk),Cumulative Duration (40wk),Cumulative Duration (60wk)
0,1,Topic 3: Data Serialization Formats - Cumulati...,4,8,12,4,8,12
1,2,Topic 4: EDA with Pandas - Cumulative Lab,5,10,15,9,18,27
2,3,Topic 5: Pandas Data Cleaning - Cumulative Lab,2,4,6,11,22,33
3,4,Topic 7: SQL - Cumulative Lab,4,8,12,15,30,45
4,5,Topic 9: APIs - Cumulative Lab,2,4,6,17,34,51
5,6,Phase 1 Project,9,18,27,26,52,78
6,7,Phase 1 Blog Post,2,4,6,28,56,84
7,8,Topic 15: Hypothesis Testing - Cumulative Lab,8,16,24,36,72,108
8,9,Topic 16: Website A/B Testing - Cumulative Lab,2,4,6,38,76,114
9,10,Topic 19: Preprocessing with scikit-learn - Cu...,7,14,21,45,90,135
